Danny Hong ECE-411 Problem Set 2: BAPM (Binomial Asset Pricing Model):

The following project consists of two classes that contain all the required functions for the Binomial Asset Pricing Model. One class deals with path-independent options while the other deals with the more general path-dependent options through performing Monte Carlo simulations. 

In [66]:
import numpy as np
import scipy.stats as ss
from IPython.display import display, Math

In [67]:
class BAPM:
  def __init__(self, r, d, u, S0, K):
    self.r = r
    self.u = u
    self.d = d
    self.S0 = S0
    self.K = K

    if S0 <= 0:
      raise ValueError("Error! The initial stock price, S0, must be positive.")

    if (1 + self.r) > self.u or (1 + self.r) < self.d:
      raise Exception("Error! There is arbitrage in the following system.")

  def get_risk_neutral_probability(self):
    risk_neutral_probability = (1 + self.r - self.d) / (self.u - self.d)
    return risk_neutral_probability

  def get_stock_price(self, num_H, num_T, S0 = None):
    if S0 == None:
      S0 = self.S0
    stock_price = S0 * (self.u ** num_H) * (self.d ** num_T)
    return stock_price

  def get_european_call_option_value(self, stock_price):
    european_call_option_value = np.maximum(0, stock_price - self.K)
    return european_call_option_value

  def get_discount_factor(self, T):
    discount_factor = 1 / ((1 + self.r) ** T)
    return discount_factor

  def path_independent_expected_value_simultation(self, T, p):
    if type(T) != int or T <= 0:
      raise ValueError("Error! The number of time steps (coin flips), T, must be a positive integer!")
    
    Sn = 0
    Vn = 0

    for i in range(T + 1): 
      stock_price = self.get_stock_price(i, T - i)
      binomial_pmf = ss.binom(T, p).pmf(i)
      european_call_option_value = self.get_european_call_option_value(stock_price)
      discount_factor = self.get_discount_factor(T)

      Sn = Sn + stock_price * binomial_pmf
      Vn = Vn + european_call_option_value * binomial_pmf

    Sn = Sn * discount_factor 
    Vn = Vn * discount_factor 

    return Sn, Vn

  def single_step_replicating_portfolio(self, VH, VT):
    equation_1 = np.array([[(1 + self.r), (self.u - (1 + self.r)) * self.S0], [(1 + self.r), (self.d - (1 + self.r)) * self.S0]])
    equation_2 = np.array([VH, VT])
    V0, delta = np.linalg.solve(equation_1, equation_2)

    return V0, delta 
  
  def multi_step_replicating_portfolio(self, N, path = []):
    if type(N) != int or N <= 0:
      raise ValueError("Error! The number of timesteps, N, must be a positive integer!")
      
    self.replicating_deltas = [None] * ((2 ** N) - 1)
    V0 = self.recursive_step_replicating_portfolio(N, path)[0]
    
    return V0, self.replicating_deltas

  def recursive_step_replicating_portfolio(self, N, path = []):
    if type(N) != int or N <= 0:
      raise ValueError("Error! The number of timesteps, N, must be a positive integer!")

    if len(path) != N:
      path_H = path + [1]
      path_T = path + [0]

    else:
      stock_price = self.get_stock_price(np.sum(path), len(path) - np.sum(path))
      european_call_option_value = self.get_european_call_option_value(stock_price) 
      return european_call_option_value, 1

    VH = self.recursive_step_replicating_portfolio(N, path_H)
    VT = self.recursive_step_replicating_portfolio(N, path_T)
      
    V0, delta = self.single_step_replicating_portfolio(VH[0], VT[0])
    
    binary_tree_index = 0
    if len(path) != 0:
      for p in path:
        binary_tree_index = binary_tree_index * 2 + p + 1

    self.replicating_deltas[binary_tree_index] = delta

    return V0, delta

In [56]:
class MC_BAPM(BAPM):
  def get_random_paths(self, p, M, N):
    random_paths = np.random.choice([0, 1], size = (M, N), p = [(1 - p), p])
    return random_paths  

  def get_lookback_option_value(self, SN, N):
    S = SN[-1]
    VN = []
    for n in range(len(SN)): 
      VN.append(((1 + self.r) ** (N - (n + 1))) * (SN[n] - S))
    lookback_option_value = np.max(VN)
    return lookback_option_value

  def path_dependent_expected_value_simultation(self, p, M, N, S0 = None, path = [], lookback_option = False):
    if type(N) != int or N <= 0:
      raise ValueError("Error! The number of timesteps, N, must be a positive integer!")

    if type(M) != int or M <= 0:
      raise ValueError("Error! The number of samples, M, must be a positive integer!")

    if S0 == None:
      S0 = self.S0

    random_paths = self.get_random_paths(p, M, N) 
    if len(path) != 0:
      random_paths = np.concatenate((np.tile(path, (M, 1)), random_paths), axis = 1)

    SN = self.get_stock_price(np.sum(random_paths, 1), random_paths.shape[1] - np.sum(random_paths, 1), S0)

    if lookback_option != False:
      V0 = self.get_lookback_option_value(SN, N)

    else:
      V0 = self.get_european_call_option_value(SN)

    SN_avg = np.mean(SN) * self.get_discount_factor(N)
    VN_avg = np.mean(V0) * self.get_discount_factor(N)

    return SN_avg, VN_avg

Exact Simulation

In [57]:
#1a.)

r = 0.25 
d = 0.5 
u = 2 
S0 = 4 
K = 5

bapm = BAPM(r, d, u, S0, K)

print("Sample Path Independent Expected Value Computation:")
print("----------------------------------------------------")

N = 1 
display(Math(r"$ N = " + str(N) + "$"))

p_tilde = bapm.get_risk_neutral_probability()
display(Math(r"$ p̃ = " + str(p_tilde) + "$"))

S0, V0 = bapm.path_independent_expected_value_simultation(N, p_tilde)
display(Math(r"$ E_p[V_N] = " + str(V0) + "$"))
print("----------------------------------------------------")

Sample Path Independent Expected Value Computation:
----------------------------------------------------


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

----------------------------------------------------


In [58]:
#1b.) 

r = 0.25 
d = 0.5 
u = 2 
S0 = 4 
K = 5

bapm = BAPM(r, d, u, S0, K)

print("Single Step Replicating Portfolio Results:")
print("-------------------------------------------")

XH_1 = bapm.get_european_call_option_value(S0 * u)
display(Math(r"$ X_1(Heads) = " + str(int(XH_1)) + r"$" )) 
XT_1 = bapm.get_european_call_option_value(S0 * d)
display(Math(r"$ X_1(Tails) = " + str(int(XT_1)) + r"$" )) 

V0, delta = bapm.single_step_replicating_portfolio(XH_1, XT_1)
display(Math(r"$ E_p[V_N] = " + str(V0) + r"$" )) 
display(Math(r"$ \Delta = " + str(delta) + r"$"))
print("-------------------------------------------")

Single Step Replicating Portfolio Results:
-------------------------------------------


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

-------------------------------------------


In [59]:
#1c.) 

r = 0.25
d = 0.5
u = 2
S0 = 4
K = 3

bapm = BAPM(r, d, u, S0, K)

print("1.) No, the deltas are not path independent.\n")
print("2.) Multi Step Replicating Portfolio Results:")

print("------------------------------------------------------------")

N = 2
display(Math(r"$ N = " + str(N) + r"$" +''))

V0, deltas = bapm.multi_step_replicating_portfolio(N)

display(Math(r"$ V_0 = " + str(V0) + r"$" )) 
display(Math(r"$ \Delta = "  + str(deltas) + r"$"))
print("------------------------------------------------------------")

1.) No, the deltas are not path independent.

2.) Multi Step Replicating Portfolio Results:
------------------------------------------------------------


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

------------------------------------------------------------


In [60]:
#1d.) 

S0 = 5
r = 0.05
u = 1.1
d = 1.01
N = 5
K = ((1 + r) ** N) * S0

bapm = BAPM(r, d, u, S0, K)

print("1.) Path Independent Expected Value Computations:")
print("-----------------------------------------------------")

p_tilde = bapm.get_risk_neutral_probability()
S0_neutral, V0_neutral = bapm.path_independent_expected_value_simultation(N, p_tilde)
display(Math(r"$ p = " + str(p_tilde) + "," + "E_p[S_N] = " + str(S0_neutral) + "," + "E_p[V_N] = " + str(V0_neutral) + r"$" ))

p_tilde_greater = p_tilde + 0.05
V0_greater, S0_greater = bapm.path_independent_expected_value_simultation(N, p_tilde_greater)
display(Math(r"$ p = " + str(p_tilde_greater) + "," + "E_p[S_N] = " + str(S0_greater) + "," + "E_p[V_N] = " + str(V0_greater) + r"$" ))

p_tilde_lower = p_tilde - 0.05
V0_lower, S0_lower = bapm.path_independent_expected_value_simultation(N, p_tilde_lower)
display(Math(r"$ p = " + str(p_tilde_lower) + "," + "E_p[S_N] = " + str(S0_lower) + "," + "E_p[V_N] = " + str(V0_lower) + r"$" ))

print("-----------------------------------------------------")
print("The correct values for S0 and V0 can be calculated through the p̃ (risk neutral probability) case.")
print("Depending on if one is buying or selling a stock, a risk premium can be observed for the other two incorrect p values.")

print("\n2.) Multi Step Replicating Portfolio Results:")
print("-----------------------------------------------------")

V0, deltas = bapm.multi_step_replicating_portfolio(N)

display(Math(r"$ V_0 = " + str(V0) + r"$" )) 
display(Math(r"$ \Delta = " + str(deltas) + r"$"))
print("-----------------------------------------------------")

print("\nNo short selling of the stock nor borrowing money from the money market need to take place along the way since all the delta values are greater than or equal to 0.")


1.) Path Independent Expected Value Computations:
-----------------------------------------------------


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

-----------------------------------------------------
The correct values for S0 and V0 can be calculated through the p̃ (risk neutral probability) case.
Depending on if one is buying or selling a stock, a risk premium can be observed for the other two incorrect p values.

2.) Multi Step Replicating Portfolio Results:
-----------------------------------------------------


<IPython.core.display.Math object>

<IPython.core.display.Math object>

-----------------------------------------------------

No short selling of the stock nor borrowing money from the money market need to take place along the way since all the delta values are greater than or equal to 0.


Monte Carlo Simulation

In [61]:
#2a.)
S0 = 1
r = 0.25
u = 1.5
d = 0.75
N = 5
K = ((1 + r) ** N) * S0
M = [1, 5, 10, 32]

mc_bapm = MC_BAPM(r, d, u, S0, K)
p_tilde = mc_bapm.get_risk_neutral_probability()

print("Sample Monte Carlo Path Dependent Expected Value Computations Where N = 5:")
print("--------------------------------------------------------------------------")

display(Math(r"$ S_0 = " + str(S0) + "$"))
p_tilde = mc_bapm.get_risk_neutral_probability()
display(Math(r"$ p̃ = " + str(p_tilde) + "$"))

SN_1, V0_1 = mc_bapm.path_dependent_expected_value_simultation(p_tilde, M[0], N)
display(Math(r"$ M = " + str(M[0]) +  "," + "S_N = " + str(SN_1) + "," + "V_0 = " + str(V0_1) + r"$"))

SN_5, V0_5 = mc_bapm.path_dependent_expected_value_simultation(p_tilde, M[1], N)
display(Math(r"$ M = " + str(M[1]) +  "," + "S_N = " + str(SN_5) + "," + "V_0 = " + str(V0_5) + r"$"))

SN_10, V0_10 = mc_bapm.path_dependent_expected_value_simultation(p_tilde, M[2], N)
display(Math(r"$ M = " + str(M[2]) +  "," + "S_N = " + str(SN_10) + "," + "V_0 = " + str(V0_10) + r"$"))

SN_32, V0_32 = mc_bapm.path_dependent_expected_value_simultation(p_tilde, M[3], N)
display(Math(r"$ M = " + str(M[3]) +  "," + "S_N = " + str(SN_32) + "," + "V_0 = " + str(V0_32) + r"$"))

print("----------------------------------------------------------------")

print("\nVerification Through the Multi-Step Replicating Portfolio Where N = 5:")
print("-------------------------------------------------------------------------")
V0, delta = mc_bapm.multi_step_replicating_portfolio(N)
display(Math(r"$ V_0 = " + str(V0) + r"$" ))
print("----------------------------------------------------------------")
print("The Monte Carlo estimates here for S0 and V0 are not that accurate since the chosen M values are rather small.")

Sample Monte Carlo Path Dependent Expected Value Computations Where N = 5:
--------------------------------------------------------------------------


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

----------------------------------------------------------------

Verification Through the Multi-Step Replicating Portfolio Where N = 5:
-------------------------------------------------------------------------


<IPython.core.display.Math object>

----------------------------------------------------------------
The Monte Carlo estimates here for S0 and V0 are not that accurate since the chosen M values are rather small.


In [62]:
#2b.)
S0 = 1
r = 10 ** -3
u = 1 + 5 * (10 ** -3)
d = 1 + (10 ** -4)
N = 10
K = ((1 + r) ** N) * S0
M = [1, 4, 64, 4096, 32768, 1048576]

mc_bapm = MC_BAPM(r, d, u, S0, K)
p_tilde = mc_bapm.get_risk_neutral_probability()

print("Sample Monte Carlo Path Dependent Expected Value Computations Where N = 10:")
print("---------------------------------------------------------------------------")

display(Math(r"$ S_0 = " + str(S0) + "$"))

p_tilde = mc_bapm.get_risk_neutral_probability()
display(Math(r"$ p̃ = " + str(p_tilde) + "$"))

SN_1, V0_1 = mc_bapm.path_dependent_expected_value_simultation(p_tilde, M[0], N)
display(Math(r"$ M = " + str(M[0]) +  "," + "S_N = " + str(SN_1) + "," + "V_0 = " + str(V0_1) + r"$"))

SN_4, V0_4 = mc_bapm.path_dependent_expected_value_simultation(p_tilde, M[1], N)
display(Math(r"$ M = " + str(M[1]) +  "," + "S_N = " + str(SN_4) + "," + "V_0 = " + str(V0_4) + r"$"))

SN_64, V0_64 = mc_bapm.path_dependent_expected_value_simultation(p_tilde, M[2], N)
display(Math(r"$ M = " + str(M[2]) +  "," + "S_N = " + str(SN_64) + "," + "V_0 = " + str(V0_64) + r"$"))

SN_4096, V0_4096 = mc_bapm.path_dependent_expected_value_simultation(p_tilde, M[3], N)
display(Math(r"$ M = " + str(M[3]) +  "," + "S_N = " + str(SN_4096) + "," + "V_0 = " + str(V0_4096) + r"$"))

SN_32768, V0_32768 = mc_bapm.path_dependent_expected_value_simultation(p_tilde, M[4], N)
display(Math(r"$ M = " + str(M[4]) +  "," + "S_N = " + str(SN_32768) + "," + "V_0 = " + str(V0_32768) + r"$"))

SN_1048576, V0_1048576 = mc_bapm.path_dependent_expected_value_simultation(p_tilde, M[5], N)
display(Math(r"$ M = " + str(M[5]) +  "," + "S_N = " + str(SN_1048576) + "," + "V_0 = " + str(V0_1048576) + r"$"))

print("----------------------------------------------------------------")

print("\nVerification Through the Multi-Step Replicating Portfolio Where N = 10:")
print("-------------------------------------------------------------------------")
V0, delta = mc_bapm.multi_step_replicating_portfolio(N)
display(Math(r"$ V_0 = " + str(V0) + r"$" ))
print("----------------------------------------------------------------")
print("The Monte Carlo estimates here for S0 and V0 are very accurate, as verified by the true V0 value obtained from the multi-step replicating portfolio process.")
print("More specifically, as M (Number of Monte Carlo Samples) increases, the calculated S0 and V0 values gets closer to the true S0 and V0 values.")

N = 100
K = ((1 + r) ** N) * S0

mc_bapm = MC_BAPM(r, d, u, S0, K)
p_tilde = mc_bapm.get_risk_neutral_probability()

print("\nSample Monte Carlo Path Dependent Expected Value Computations Where N = 100:")
print("------------------------------------------------------------------------------")

display(Math(r"$ S_0 = " + str(S0) + "$"))

p_tilde = mc_bapm.get_risk_neutral_probability()
display(Math(r"$ p̃ = " + str(p_tilde) + "$"))

SN_1, V0_1 = mc_bapm.path_dependent_expected_value_simultation(p_tilde, M[0], N)
display(Math(r"$ M = " + str(M[0]) +  "," + "S_N = " + str(SN_1) + "," + "V_0 = " + str(V0_1) + r"$"))

SN_4, V0_4 = mc_bapm.path_dependent_expected_value_simultation(p_tilde, M[1], N)
display(Math(r"$ M = " + str(M[1]) +  "," + "S_N = " + str(SN_4) + "," + "V_0 = " + str(V0_4) + r"$"))

SN_64, V0_64 = mc_bapm.path_dependent_expected_value_simultation(p_tilde, M[2], N)
display(Math(r"$ M = " + str(M[2]) +  "," + "S_N = " + str(SN_64) + "," + "V_0 = " + str(V0_64) + r"$"))

SN_4096, V0_4096 = mc_bapm.path_dependent_expected_value_simultation(p_tilde, M[3], N)
display(Math(r"$ M = " + str(M[3]) +  "," + "S_N = " + str(SN_4096) + "," + "V_0 = " + str(V0_4096) + r"$"))

SN_32768, V0_32768 = mc_bapm.path_dependent_expected_value_simultation(p_tilde, M[4], N)
display(Math(r"$ M = " + str(M[4]) +  "," + "S_N = " + str(SN_32768) + "," + "V_0 = " + str(V0_32768) + r"$"))

SN_1048576, V0_1048576 = mc_bapm.path_dependent_expected_value_simultation(p_tilde, M[5], N)
display(Math(r"$ M = " + str(M[5]) +  "," + "S_N = " + str(SN_1048576) + "," + "V_0 = " + str(V0_1048576) + r"$"))

print("------------------------------------------------------------------------")

Sample Monte Carlo Path Dependent Expected Value Computations Where N = 10:
---------------------------------------------------------------------------


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

------------------------------------------------------------------------


In [63]:
#2c.) 
S0 = 1
r = 10 ** -3
u = 1 + 5 * (10 ** -3)
d = 1 + (10 ** -4)
N = 100
K = ((1 + r) ** N) * S0
M = [4, 16, 256, 65536, 1048576]

mc_bapm = MC_BAPM(r, d, u, S0, K)

p_tilde = mc_bapm.get_risk_neutral_probability()
p = 0.9 * p_tilde

num_paths = 5
path_length = 10
random_paths = np.random.choice([0, 1], size = (num_paths, path_length), p = [(1 - p), p])

print("1.) Monte Carlo Method for Multiple Paths and Multiple M (Number of Samples) Values Simulation Results Where N = 100:")
print("---------------------------------------------------------------------------------------------------------------------")

error_list = []
for i in range (len(random_paths)):
  path_price = mc_bapm.get_stock_price(np.sum(random_paths[i]), len(random_paths[i]) - np.sum(random_paths[i]))
  for M_value in M:
    SN, V0 = mc_bapm.path_dependent_expected_value_simultation(p, M_value, N, S0, random_paths[i])
    error = path_price - SN
    error_list.append(error)
    print("Path # = " + str(i + 1) + ", " + "M = " + str(M_value) + ", " + "True S10 = " + str(path_price) + ", " + "Sn = " + str(SN) + ", " + "V10 = " + str(V0) + ", " + "S10 - Sn = " + str(error))

print("\nAverage Difference (S0 - Sn) = " + str(np.mean(error_list)))
print("---------------------------------------------------------------------------------------------------------")
print("The Monte Carlo estimates for SN is very accurate as the average difference between the estimated SN and true S0 values is shown to be relatively small.")

mc_bapm = MC_BAPM(r, d, u, S0, K)

p_tilde = mc_bapm.get_risk_neutral_probability()
p = 0.9 * p_tilde

print("\n2.) Monte Carlo Method for Fixed Path and Fixed M (Number of Samples) Values Simulation Results Where N = 100:")
print("----------------------------------------------------------------------------------------------------------------")

fixed_path = random_paths[0]
path_price = mc_bapm.get_stock_price(np.sum(fixed_path), len(fixed_path) - np.sum(fixed_path))
fixed_M = M[4]
num_iterations = 50

SN_list = []
error_list = []
for j in range(num_iterations):
  SN, V0 = mc_bapm.path_dependent_expected_value_simultation(p, fixed_M, N, S0, fixed_path)
  error = path_price - SN
  error_list.append(error)
  print("Iteration # = " + str(j + 1) + ", " + "M = " + str(fixed_M) + ", " + "True S10 = " + str(path_price) + ", " + "Sn = " + str(SN) + ", " + "V10 = " + str(V0) + ", " + "S10 - Sn = " + str(error))
  SN_list.append(np.mean(SN))

print("\nMean of the Sn values = " + str(np.mean(SN_list)) + ", " + "Variance of the Sn Values = " + str(np.var(SN_list, ddof = 1)) + ", " + "Average Difference (S10 - Sn) = " + str(np.mean(error_list)))
print("----------------------------------------------------------------------------------------------------")

print("The Monte Carlo estimates for SN here are shown to be consistently accurate since the mean of all the estimated SN values is very close to the true S10 value.")
print("In addition, the variance of the estimated SN values is also very low, which also shows that the results are consistently close in value to each other.")

1.) Monte Carlo Method for Multiple Paths and Multiple M (Number of Samples) Values Simulation Results Where N = 100:
---------------------------------------------------------------------------------------------------------------------
Path # = 1, M = 4, True S10 = 1.0059048618844324, Sn = 0.9990793670714905, V10 = 0.00441959191902053, S10 - Sn = 0.0068254948129419235
Path # = 1, M = 16, True S10 = 1.0059048618844324, Sn = 0.9951698459036299, V10 = 0.004146978267292171, S10 - Sn = 0.01073501598080251
Path # = 1, M = 256, True S10 = 1.0059048618844324, Sn = 0.9965792003744915, V10 = 0.005629158445265001, S10 - Sn = 0.009325661509940852
Path # = 1, M = 65536, True S10 = 1.0059048618844324, Sn = 0.9969899028651942, V10 = 0.005845954974259052, S10 - Sn = 0.008914959019238222
Path # = 1, M = 1048576, True S10 = 1.0059048618844324, Sn = 0.9968802520785419, V10 = 0.005827858444867362, S10 - Sn = 0.009024609805890482
Path # = 2, M = 4, True S10 = 1.001000450120021, Sn = 0.9894670582169596, V10

In [64]:
#2d Part 1: Repeat 2b.)
S0 = 1
r = 10 ** -3
u = 1 + (5 * (10 ** -3))
d = 1 + (10 ** -4)
N = 100
K = ((1 + r) ** N) * S0
M = [1, 4, 64, 4096, 32768, 1048576]

mc_bapm = MC_BAPM(r, d, u, S0, K)
p_tilde = mc_bapm.get_risk_neutral_probability()

print("Sample Monte Carlo Path Dependent Expected Value Computations Using the Lookback Option Where N = 100:")
print("------------------------------------------------------------------------------------------------------")

display(Math(r"$ S_0 = " + str(S0) + "$"))

p_tilde = mc_bapm.get_risk_neutral_probability()
display(Math(r"$ p̃ = " + str(p_tilde) + "$"))

SN_1, V0_1 = mc_bapm.path_dependent_expected_value_simultation(p_tilde, M[0], N, lookback_option = True)
display(Math(r"$ M = " + str(M[0]) +  "," + "S_N = " + str(SN_1) + "," + "V_0 = " + str(V0_1) + r"$"))

SN_4, V0_4 = mc_bapm.path_dependent_expected_value_simultation(p_tilde, M[1], N, lookback_option = True)
display(Math(r"$ M = " + str(M[1]) +  "," + "S_N = " + str(SN_4) + "," + "V_0 = " + str(V0_4) + r"$"))

SN_64, V0_64 = mc_bapm.path_dependent_expected_value_simultation(p_tilde, M[2], N, lookback_option = True)
display(Math(r"$ M = " + str(M[2]) +  "," + "S_N = " + str(SN_64) + "," + "V_0 = " + str(V0_64) + r"$"))

SN_4096, V0_4096 = mc_bapm.path_dependent_expected_value_simultation(p_tilde, M[3], N, lookback_option = True)
display(Math(r"$ M = " + str(M[3]) +  "," + "S_N = " + str(SN_4096) + "," + "V_0 = " + str(V0_4096) + r"$"))

SN_32768, V0_32768 = mc_bapm.path_dependent_expected_value_simultation(p_tilde, M[4], N, lookback_option = True)
display(Math(r"$ M = " + str(M[4]) +  "," + "S_N = " + str(SN_32768) + "," + "V_0 = " + str(V0_32768) + r"$"))

SN_1048576, V0_1048576 = mc_bapm.path_dependent_expected_value_simultation(p_tilde, M[5], N, lookback_option = True)
display(Math(r"$ M = " + str(M[5]) +  "," + "S_N = " + str(SN_1048576) + "," + "V_0 = " + str(V0_1048576) + r"$"))

print("------------------------------------------------------------------------")
print("The lookback option is shown to be effective since the estimated SN values get closer to the true S0 value as M gets larger.")

Sample Monte Carlo Path Dependent Expected Value Computations Using the Lookback Option Where N = 100:
------------------------------------------------------------------------------------------------------


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

------------------------------------------------------------------------
The lookback option is shown to be effective since the estimated SN values get closer to the true S0 value as M gets larger.


In [65]:
#2d Part 2: Repeat 2c.)
S0 = 1
r = 10 ** -3
u = 1 + (5 * (10 ** -3))
d = 1 + (10 ** -4)
N = 100
K = ((1 + r) ** N) * S0
M = [4, 16, 256, 65536, 1048576]

mc_bapm = MC_BAPM(r, d, u, S0, K)

p_tilde = mc_bapm.get_risk_neutral_probability()
p = 0.9 * p_tilde

num_paths = 5
path_length = 10
random_paths = np.random.choice([0, 1], size = (num_paths, path_length), p = [(1 - p), p])

print("1.) Monte Carlo Method Using the Lookback Option for Multiple Paths and Multiple M (Number of Samples) Values Simulation Results Where N = 100:")
print("-----------------------------------------------------------------------------------------------------------------------------------------------")

error_list = []
for i in range (len(random_paths)):  
  path_price = mc_bapm.get_stock_price(np.sum(random_paths[i]), len(random_paths[i]) - np.sum(random_paths[i]))
  stock_price = S0
  for n in range(path_length):
    if random_paths[i][n] == 1:
      stock_price = stock_price * (u)
    else:
      stock_price = stock_price * (d)
    max_value = np.maximum(0, stock_price * ((1 + r) ** (path_length - (n + 1))))

  for M_value in M:
    SN, V0 = mc_bapm.path_dependent_expected_value_simultation(p, M_value, N, S0, random_paths[i], lookback_option = True)
    error = path_price - SN
    error_list.append(error)
    print("Path # = " + str(i + 1) + ", " + "M = " + str(M_value) + ", " + "True S10 = " + str(path_price) + ", " + "True M10 = " + str(max_value) + ", " + "Sn = " + str(SN) + ", " + "V10 = " + str(V0) + ", " + "S10 - Sn = " + str(error) + ", ")

print("\nAverage Difference (S0 - Sn) = " + str(np.mean(error_list)))
print("---------------------------------------------------------------------------------------------------------------------------------------------")
print("The Monte Carlo estimates from using the Lookback Option for SN is very accurate as the average difference between the estimated SN and true S0 values is shown to be relatively small.")

mc_bapm = MC_BAPM(r, d, u, S0, K)

p_tilde = mc_bapm.get_risk_neutral_probability()
p = 0.9 * p_tilde

print("\n2.) Monte Carlo Method from using the Lookback Option for Fixed Path and Fixed M (Number of Samples) Values Simulation Results Where N = 100:")
print("-----------------------------------------------------------------------------------------------------------------------------------------------")

fixed_path = random_paths[0]
path_price = mc_bapm.get_stock_price(sum(fixed_path), len(fixed_path) - sum(fixed_path))
fixed_M = M[4]
num_iterations = 50

stock_price = S0
for n in range(path_length):
  if fixed_path[n] == 1:
    stock_price = stock_price * (u)
  else:
    stock_price = stock_price * (d)
  max_value = np.maximum(0, stock_price * ((1 + r) ** (path_length - (n + 1))))

SN_list = []
error_list = []
for j in range(num_iterations):
  SN, V0 = mc_bapm.path_dependent_expected_value_simultation(p, fixed_M, N, S0, fixed_path, lookback_option = True)
  error = path_price - SN
  error_list.append(error)
  print("Iteration # = " + str(j + 1) + ", " + "M = " + str(fixed_M) + ", " + "True S10 = " + str(path_price) + ", " + "True M10 = " + str(max_value) + ", " + "Sn = " + str(SN) + ", " + "V10 = " + str(V0) + ", " + "S10 - Sn = " + str(error))
  SN_list.append(np.mean(SN))

print("\nMean of the Sn values = " + str(np.mean(SN_list)) + ", " + "Variance of the Sn Values = " + str(np.var(SN_list, ddof = 1)) + ", " + "Average Difference (S0 - Sn) = " + str(np.mean(error_list)))
print("----------------------------------------------------------------------------------------------------")

print("The Monte Carlo estimates for SN from using the lookback option here are shown to be consistently accurate since the mean of all the estimated SN values is very close to the true S0 value.")
print("In addition, the variance of the estimated SN values is also very low, which also shows that the results are consistently close in value to each other.")
print("The lookback option satisfies the martingale property, so therefore, the value of the option depends on the minimum price of the underlying asset over a specified period, which is a decreasing function of time.")
print("Therefore, as shown through the results where the True S10 value is extremely close to the True M10 value, the expected value of the option at the time of maturity is equal to its current value.")

1.) Monte Carlo Method Using the Lookback Option for Multiple Paths and Multiple M (Number of Samples) Values Simulation Results Where N = 100:
-----------------------------------------------------------------------------------------------------------------------------------------------
Path # = 1, M = 4, True S10 = 1.010833302863568, True M10 = 1.010833302863568, Sn = 0.9991934918871083, V10 = 0.05406783160279852, S10 - Sn = 0.01163981097645983, 
Path # = 1, M = 16, True S10 = 1.010833302863568, True M10 = 1.010833302863568, Sn = 1.007178110195915, V10 = 0.04375536352468498, S10 - Sn = 0.0036551926676531288, 
Path # = 1, M = 256, True S10 = 1.010833302863568, True M10 = 1.010833302863568, Sn = 1.0038791040185198, V10 = 0.11220271101727892, S10 - Sn = 0.006954198845048332, 
Path # = 1, M = 65536, True S10 = 1.010833302863568, True M10 = 1.010833302863568, Sn = 1.0017767822069612, V10 = 0.02356755707497726, S10 - Sn = 0.009056520656606892, 
Path # = 1, M = 1048576, True S10 = 1.01083330